In [ ]:
import os
import sys
import glob
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, OrderedDict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

NFOLDS = 5
BATCH_SIZE = 128
LR = 1e-3
EPOCHS = 50
EARLYSTOP_NUM = 3
SCHEDULE_DECAY = 2

DEBUG = False
if DEBUG:
    NFOLDS = 2
    EPOCHS = 2

feature_dir = "../input/indoor-navigation-and-location-wifi-features"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
# print('subm:\n', subm.head())

save_path = './'
if not os.path.exists(save_path):
    os.mkdir(save_path)
if DEBUG:
    save_path = f'{save_path}/debug'
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    train_files = train_files[:3]

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            # print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            # print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            # if not DEBUG:
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

##### Model Fn
class IndoorDataset:
    def __init__(self, feat, x, y, f):
        self.feat = feat
        self.label_x = x.reshape(-1, 1)
        self.label_y = y.reshape(-1, 1)
        self.label_f = f.reshape(-1, 1)

    def __len__(self):
        return len(self.feat)

    def __getitem__(self, idx):
        return {
            'feat': torch.tensor(self.feat[idx], dtype=torch.float),

            'label_x': torch.tensor(self.label_x[idx], dtype=torch.float),
            'label_y': torch.tensor(self.label_y[idx], dtype=torch.float),
            'label_f': torch.tensor(self.label_f[idx], dtype=torch.float),
        }

class MeanPositionLoss(nn.Module):
    def __init__(self):
        super(MeanPositionLoss, self).__init__()
    def forward(self, output_way_x, output_way_y, output_floor, way_x, way_y, floor):
        diff_x = output_way_x - way_x
        diff_y = output_way_y - way_y
        diff_f = output_floor - floor

        error = torch.sqrt(diff_x * diff_x + diff_y * diff_y) + 15 * torch.sqrt(diff_f * diff_f)
        return torch.mean(error)

def MeanPositionScore(pred_x, pred_y, pred_f, label_x, label_y, label_f):
    diff_f = pred_f - label_f
    diff_x = pred_x - label_x
    diff_y = pred_y - label_y
    error = np.sqrt(diff_x * diff_x + diff_y * diff_y) + 15 * np.sqrt(diff_f * diff_f)
    return np.mean(error)

def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = AverageMeter()

    # tk0 = tqdm(data_loader, total=len(data_loader), desc="Train")
    for bi, d in enumerate(data_loader):
        torch.cuda.empty_cache()
        feat = d['feat']

        label_x = d["label_x"]
        label_y = d['label_y']
        label_f = d['label_f']

        feat = feat.to(device, dtype=torch.float)

        label_x = label_x.to(device, dtype=torch.float)
        label_y = label_y.to(device, dtype=torch.float)
        label_f = label_f.to(device, dtype=torch.float)

        optimizer.zero_grad()
        output_x, output_y, output_f = model(
            x=feat,
        )
        torch.cuda.empty_cache()
        loss = loss_fn(output_x, output_y, output_f, label_x, label_y, label_f)
        torch.cuda.empty_cache()
        loss.backward()
        optimizer.step()
        # ema.update()
        if scheduler:
            scheduler.step()

        torch.cuda.empty_cache()
        losses.update(loss.item(), feat.size(0))
        # tk0.set_postfix(loss=losses.avg)

def infer_fn(data_loader, model, device):
    model.eval()
    pred_x, pred_y, pred_f = [], [], []

    # tk0 = tqdm(data_loader, total=len(data_loader), desc="Infer")
    with torch.no_grad():
        for bi, d in enumerate(data_loader):
            feat = d['feat']

            feat = feat.to(device, dtype=torch.float)

            output_x, output_y, output_f = model(
                x=feat,
            )

            pred_x.extend(output_x.cpu().detach().numpy())
            pred_y.extend(output_y.cpu().detach().numpy())
            pred_f.extend(output_f.cpu().detach().numpy())
    # print('pred_f: ', pred_f.shape)

    return np.concatenate(pred_x), np.concatenate(pred_y), np.concatenate(pred_f)

class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(input_dim)
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(input_dim, hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size, hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.regressioner = nn.Linear(hidden_size, 3)

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x1 = F.relu(x1)
        # x1 = self.PReLU(x1)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x2 = self.dense2(x1)
        x2 = self.batch_norm2(x2)
        # x2 = F.relu(x2)
        # x2 = self.PReLU(x2)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x3 = self.dense3(x2)
        x3 = self.batch_norm3(x3)
        # x3 = F.relu(x3)
        # x3 = self.PReLU(x3)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        output = self.regressioner(x3)
        output_x, output_y, output_f = output.split(1, dim=-1)
        # print('output_x: ', output_x.shape)
        # print('output_y: ', output_y.shape)
        # print('output_f: ', output_f.shape)
        return output_x, output_y, output_f

all_oof, all_pred = [], []
start_time = time.time()
for n_files, file in enumerate(train_files):
    seed_everything(seed=42)

    SiteID = file.split('/')[-1].split('_')[0]
    train = pd.read_csv(file, index_col=0)
    # print(train.head())

    groups = train.iloc[:, -1].values
    train_feat, train_x, train_y, train_f = train.iloc[:, :-4].values, train.iloc[:, -4].values, train.iloc[:, -3].values, train.iloc[:, -2].values
    feat_dim = train_feat.shape[1]

    train_feat[train_feat == -999] = -99
    train_feat = train_feat / 100. + 1.

    print(f'[{n_files+1:2}/{len(train_files)}]\t{SiteID} Train: {len(train):5}\tFeat Num: {feat_dim:5}')

    oof_x, oof_y, oof_f = np.zeros(len(train_feat)), np.zeros(len(train_feat)), np.zeros(len(train_feat))
    # kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)
    kf = GroupKFold(n_splits=NFOLDS)
    for _fold, (trn_idx, val_idx) in enumerate(kf.split(train_feat, groups=groups)):
        # if _fold != 4:
        #     continue
        train_set = IndoorDataset(
            feat=train_feat[trn_idx],
            x=train_x[trn_idx],
            y=train_y[trn_idx],
            f=train_f[trn_idx],
        )
        valid_set = IndoorDataset(
            feat=train_feat[val_idx],
            x=train_x[val_idx],
            y=train_y[val_idx],
            f=train_f[val_idx],
        )
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=4)
        valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        torch.cuda.empty_cache()
        device = torch.device('cuda')
        model = Model(input_dim=feat_dim)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,
        #                                                        patience=SCHEDULE_DECAY, verbose=False)
        scheduler = None

        loss_fn = MeanPositionLoss()

        best_mpe = 500.
        model_weights = f"{save_path}/{SiteID}_model{_fold}.pth"
        es = EarlyStopping(patience=EARLYSTOP_NUM, mode="min")
        for epoch in range(EPOCHS):
            train_fn(train_loader, model, optimizer, device, scheduler=None)
            val_pred_x, val_pred_y, val_pred_f = infer_fn(valid_loader, model, device)
            val_mpe = MeanPositionScore(val_pred_x, val_pred_y, val_pred_f, train_x[val_idx], train_y[val_idx], train_f[val_idx])
            if np.isnan(val_mpe):
                break

            if best_mpe >= val_mpe:
                best_mpe = val_mpe
            if scheduler:
                scheduler.step(best_mpe)

            # print(f"\tFOLD{_fold} EPOCH:{epoch:3} val_mpe={val_mpe:.5f} "
            #       f"time: {(time.time() - start_time) / 60:.2f}min")

            es(val_mpe, model, model_path=model_weights)
            if es.early_stop:
                # print("Early stopping")
                break
        # val&infer pred
        model.load_state_dict(torch.load(model_weights))
        oof_x[val_idx], oof_y[val_idx], oof_f[val_idx] = infer_fn(valid_loader, model, device)
    oof_mpe = MeanPositionScore(oof_x, oof_y, oof_f, train_x, train_y, train_f)
    print(f'\t{SiteID} OOF MPE: {oof_mpe:.5f} time: {(time.time() - start_time) / 60:.2f}min')

    train_pred_df = train[['path']].copy(deep=True)
    train_pred_df['label_floor'] = train_f
    train_pred_df['label_x'] = train_x
    train_pred_df['label_y'] = train_y
    train_pred_df['floor'] = oof_f
    train_pred_df['x'] = oof_x
    train_pred_df['y'] = oof_y
    all_oof.append(train_pred_df)

all_oof = pd.concat(all_oof).reset_index(drop=True)
all_oof['floor'] = all_oof['floor'].apply(lambda x: int(x))
all_oof_mpe = MeanPositionScore(all_oof.x.values, all_oof.y.values, all_oof.floor.values, all_oof.label_x.values, all_oof.label_y.values, all_oof.label_floor.values)
print(f'\nAll OOF MPE: {all_oof_mpe:.4f}')
all_oof.to_csv(f'{save_path}/all_oof.csv', index=False)
